**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Sun May 30 13:35:53 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: failed to create symbolic link '/mydrive/My Drive': File exists
 0e3f6880-c08c-4c9b-a411-767f404c50c8.JPG
 18BCE2330_CAT1.pdf
'18BCE2330_Dhananjay Sunil Menon_DA2.pdf'
'18BCE2330 Guest Lecture 2.pdf'
 18BCE2330-ISAA.pdf
 18BCE2330_LabDA06.pdf
'18BCE2330  - T E1 - GUEST LECTURE 1 .pdf'
 18BCE2330_WebMiningDa2
'1. About Us.psd'
 aff1b7ad-012f-4b3c-ac29-e8aedfe40278.MP4
'Annlyn 2 - Priceless Senior Citizens.psd'
'Annlyn 3 - Priceless Senior Citizens.psd'
'Annlyn - final.psd'
'Annlyn - Priceless Senior Citizens.psd'
'Arnab 2 - The Game that beat the pandemic.psd'
'Arnab - The Game that beat the pandemic.psd'
'Artificial Intelligence - Cat 2 Papers'
'Arun 2 - Title.psd'
'Arun - Title.psd'
 Auh_IP
'AUH roads'
 b41c5bc5-c386-4318-9214-827f6d1044c8.JPG
'Buffina 2 - Kindness.psd'
'Buffina - Kindness.psd'
 Buffina.psd
'Colab Notebooks'
 Concepts_of_Modern_Physics

**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15054, done.
remote: Total 15054 (delta 0), reused 0 (delta 0), pack-reused 15054
Receiving objects: 100% (15054/15054), 13.43 MiB | 15.30 MiB/s, done.
Resolving deltas: 100% (10234/10234), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:935:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1136:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [ ]:
!echo "car" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-05-30 13:37:04--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  23.3MB/s    in 7.4s    

2021-05-30 13:37:12 (20.9 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/00001.jpg      
  inflating: data/obj/00001.txt      
  inflating: data/obj/00002.jpg      
  inflating: data/obj/00002.txt      
  inflating: data/obj/00003.jpg      
  inflating: data/obj/00003.txt      
  inflating: data/obj/00004.jpg      
  inflating: data/obj/00004.txt      
  inflating: data/obj/00005.jpg      
  inflating: data/obj/00005.txt      
  inflating: data/obj/00006.jpg      
  inflating: data/obj/00006.txt      
  inflating: data/obj/00007.jpg      
  inflating: data/obj/00007.txt      
  inflating: data/obj/00008.jpg      
  inflating: data/obj/00008.txt      
  inflating: data/obj/00009.jpg      
  inflating: data/obj/00009.txt      
  inflating: data/obj/00010.jpg      
  inflating: data/obj/00010.txt      
  inflating: data/obj/00011.jpg      
  inflating: data/obj/00011.txt      
  inflating: data/obj/00012.jpg      
  inflating: data/obj/00012.txt      
  inflating: data/obj/00013.jpg      
  inflating: 

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

4 0.498182 0.533223 0.967273 0.734219

['4', '0.498182', '0.533223', '0.967273', '0.734219']
0 data/obj/00081.txt
0 0.498182 0.533223 0.967273 0.734219
4 0.242188 0.564453 0.382812 0.311198

['4', '0.242188', '0.564453', '0.382812', '0.311198']
1 data/obj/00031.txt
0 0.242188 0.564453 0.382812 0.311198
4 0.521000 0.548000 0.774000 0.776000

['4', '0.521000', '0.548000', '0.774000', '0.776000']
2 data/obj/111712537.txt
0 0.521000 0.548000 0.774000 0.776000
4 0.514583 0.571094 0.937500 0.629687

['4', '0.514583', '0.571094', '0.937500', '0.629687']
3 data/obj/00203.txt
0 0.514583 0.571094 0.937500 0.629687
4 0.513184 0.542318 0.926758 0.738281

['4', '0.513184', '0.542318', '0.926758', '0.738281']
4 data/obj/00159.txt
0 0.513184 0.542318 0.926758 0.738281
4 0.542063 0.744048 0.795238 0.407143

['4', '0.542063', '0.744048', '0.795238', '0.407143']
5 data/obj/00118.txt
0 0.542063 0.744048 0.795238 0.407143
4 0.505078 0.563477 0.910156 0.509766

['4', '0.505078', '0.563477', '0.910156', '0.

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/00132.jpg', 'data/obj/00104.jpg', 'data/obj/00116.jpg', 'data/obj/00412.jpg', 'data/obj/00068.jpg', 'data/obj/00078.jpg', 'data/obj/00320.jpg', 'data/obj/00285.jpg', 'data/obj/00137.jpg', 'data/obj/00143.jpg', 'data/obj/00218.jpg', 'data/obj/00065.jpg', 'data/obj/00397.jpg', 'data/obj/00021.jpg', 'data/obj/00325.jpg', 'data/obj/00081.jpg', 'data/obj/00054.jpg', 'data/obj/00012.jpg', 'data/obj/714477016.jpg', 'data/obj/00095.jpg', 'data/obj/00062.jpg', 'data/obj/00236.jpg', 'data/obj/00245.jpg', 'data/obj/00026.jpg', 'data/obj/00371.jpg', 'data/obj/00119.jpg', 'data/obj/00020.jpg', 'data/obj/00047.jpg', 'data/obj/00250.jpg', 'data/obj/00090.jpg', 'data/obj/00193.jpg', 'data/obj/00179.jpg', 'data/obj/00106.jpg', 'data/obj/00388.jpg', 'data/obj/00434.jpg', 'data/obj/00270.jpg', 'data/obj/00281.jpg', 'data/obj/00170.jpg', 'data/obj/00017.jpg', 'data/obj/00192.jpg', 'data/obj/00109.jpg', 'data/obj/00159.jpg', 'data/obj/00426.jpg', 'data/obj/00284.jpg', 'data/obj/00124.jpg', 'data

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
 total_bbox = 265398, rewritten_bbox = 0.019216 % 

 3926: 0.022889, 0.026604 avg loss, 0.001000 rate, 5.317160 seconds, 251264 images, 0.158436 hours left
Loaded: 0.000072 seconds
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.879016), count: 4, class_loss = 0.005089, iou_loss = 0.054006, total_loss = 0.059095 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 265402, rewritten_bbox = 0.019216 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.901800), count: 4, class_loss = 0.000003, iou_loss = 0.024124, total_loss = 0.024127 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1